In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(
    "in_rolls_state_name_age_sex_fn_ln.csv.gz",
    usecols=["state", "birth_year", "first_name", "sex"],
)
df

,sex,state,birth_year,first_name
0,male,andaman,1979.0,pintu
1,female,andaman,1978.0,sushila
2,male,andaman,1988.0,bala
3,male,andaman,1972.0,chandra
4,male,andaman,1968.0,sikander
...,...,...,...,...
562652713,male,uttarakhand,1983.0,vivek
562652714,female,uttarakhand,1986.0,renoo
562652715,male,uttarakhand,1986.0,abhinav
562652716,female,uttarakhand,1987.0,priyamkaa


In [3]:
# V2: Clean non-alpha from first_name
df["first_name"] = df.first_name.str.replace("[^a-zA-Z]", "", regex=True).str.strip()
df

,sex,state,birth_year,first_name
0,male,andaman,1979.0,pintu
1,female,andaman,1978.0,sushila
2,male,andaman,1988.0,bala
3,male,andaman,1972.0,chandra
4,male,andaman,1968.0,sikander
...,...,...,...,...
562652713,male,uttarakhand,1983.0,vivek
562652714,female,uttarakhand,1986.0,renoo
562652715,male,uttarakhand,1986.0,abhinav
562652716,female,uttarakhand,1987.0,priyamkaa


In [4]:
df.dropna(subset=["first_name"], inplace=True)
df

,sex,state,birth_year,first_name
0,male,andaman,1979.0,pintu
1,female,andaman,1978.0,sushila
2,male,andaman,1988.0,bala
3,male,andaman,1972.0,chandra
4,male,andaman,1968.0,sikander
...,...,...,...,...
562652712,female,uttarakhand,1959.0,sarooj
562652713,male,uttarakhand,1983.0,vivek
562652714,female,uttarakhand,1986.0,renoo
562652715,male,uttarakhand,1986.0,abhinav


* i think it makes sense to only release data where count for last_name/first_name is > 100
* it will take out a bunch of weird names for now

In [5]:
adf = df[df["first_name"].groupby(df["first_name"]).transform("size") > 100]
adf

,sex,state,birth_year,first_name
0,male,andaman,1979.0,pintu
1,female,andaman,1978.0,sushila
2,male,andaman,1988.0,bala
3,male,andaman,1972.0,chandra
4,male,andaman,1968.0,sikander
...,...,...,...,...
562652712,female,uttarakhand,1959.0,sarooj
562652713,male,uttarakhand,1983.0,vivek
562652714,female,uttarakhand,1986.0,renoo
562652715,male,uttarakhand,1986.0,abhinav


In [6]:
adf["first_name"].groupby(adf["first_name"]).size()

first_name
              4244
aa           21192
aaa            319
aaaa           136
aaaamsoo      2036
             ...  
zulfikar       174
zulfiqar       102
zungkum        137
zuthunglo      184
zv             255
Name: first_name, Length: 197345, dtype: int64

In [7]:
adf = adf[adf.first_name.str.contains("[a-z]")]
adf = adf[~adf.first_name.str.contains(r"\.")]
adf

,sex,state,birth_year,first_name
0,male,andaman,1979.0,pintu
1,female,andaman,1978.0,sushila
2,male,andaman,1988.0,bala
3,male,andaman,1972.0,chandra
4,male,andaman,1968.0,sikander
...,...,...,...,...
562652712,female,uttarakhand,1959.0,sarooj
562652713,male,uttarakhand,1983.0,vivek
562652714,female,uttarakhand,1986.0,renoo
562652715,male,uttarakhand,1986.0,abhinav


In [8]:
gdf = (
    adf.groupby(["state", "birth_year", "first_name"])
    .sex.value_counts()
    .unstack(fill_value=0)
)

In [9]:
gdf

sex                                  female  male  third gender
state       birth_year first_name                              
andaman     1898.0     hiramani           1     0             0
                       sabha              1     0             0
            1902.0     samir              0     1             0
            1904.0     siri               0     1             0
            1905.0     cyril              0     1             0
...                                     ...   ...           ...
uttarakhand 2016.0     syaam              0     1             0
                       ushaa              1     0             0
            2017.0     chandrakalee       1     0             0
                       kiranavathee       1     0             0
                       samjay             0     1             0

[23824378 rows x 3 columns]

In [10]:
gdf.reset_index(inplace=True)
gdf

sex,state,birth_year,first_name,female,male,third gender
0,andaman,1898.0,hiramani,1,0,0
1,andaman,1898.0,sabha,1,0,0
2,andaman,1902.0,samir,0,1,0
3,andaman,1904.0,siri,0,1,0
4,andaman,1905.0,cyril,0,1,0
...,...,...,...,...,...,...
23824373,uttarakhand,2016.0,syaam,0,1,0
23824374,uttarakhand,2016.0,ushaa,1,0,0
23824375,uttarakhand,2017.0,chandrakalee,1,0,0
23824376,uttarakhand,2017.0,kiranavathee,1,0,0


In [11]:
gdf.columns = [
    "state",
    "birth_year",
    "first_name",
    "n_female",
    "n_male",
    "n_third_gender",
]
gdf

,state,birth_year,first_name,n_female,n_male,n_third_gender
0,andaman,1898.0,hiramani,1,0,0
1,andaman,1898.0,sabha,1,0,0
2,andaman,1902.0,samir,0,1,0
3,andaman,1904.0,siri,0,1,0
4,andaman,1905.0,cyril,0,1,0
...,...,...,...,...,...,...
23824373,uttarakhand,2016.0,syaam,0,1,0
23824374,uttarakhand,2016.0,ushaa,1,0,0
23824375,uttarakhand,2017.0,chandrakalee,1,0,0
23824376,uttarakhand,2017.0,kiranavathee,1,0,0


In [12]:
gdf["prop_female"] = gdf["n_female"] / (
    gdf["n_female"] + gdf["n_male"] + gdf["n_third_gender"]
)
gdf

,state,birth_year,first_name,n_female,n_male,n_third_gender,prop_female
0,andaman,1898.0,hiramani,1,0,0,1.0
1,andaman,1898.0,sabha,1,0,0,1.0
2,andaman,1902.0,samir,0,1,0,0.0
3,andaman,1904.0,siri,0,1,0,0.0
4,andaman,1905.0,cyril,0,1,0,0.0
...,...,...,...,...,...,...,...
23824373,uttarakhand,2016.0,syaam,0,1,0,0.0
23824374,uttarakhand,2016.0,ushaa,1,0,0,1.0
23824375,uttarakhand,2017.0,chandrakalee,1,0,0,1.0
23824376,uttarakhand,2017.0,kiranavathee,1,0,0,1.0


In [13]:
gdf.to_csv("in_rolls_state_year_fn_naampy.csv.gz", index=False, compression="gzip")

In [14]:
bdf = (
    gdf.groupby(["first_name"])
    .agg({"n_female": "sum", "n_male": "sum", "n_third_gender": "sum"})
    .reset_index()
)
bdf["prop_female"] = bdf["n_female"] / (
    bdf["n_female"] + bdf["n_male"] + gdf["n_third_gender"]
)
bdf

,first_name,n_female,n_male,n_third_gender,prop_female
0,aa,948,20244,0,0.044734
1,aaa,83,236,0,0.260188
2,aaaa,101,35,0,0.742647
3,aaaamsoo,1375,661,0,0.675344
4,aaaan,3,100,0,0.029126
...,...,...,...,...,...
197339,zulfikar,0,174,0,0.000000
197340,zulfiqar,0,102,0,0.000000
197341,zungkum,0,137,0,0.000000
197342,zuthunglo,182,2,0,0.989130


In [ ]:
# Resource List: procs=1,mem=200gb
# Resources Used: cput=00:40:35,vmem=135071932kb,walltime=00:41:25,mem=143391788kb,energy_used=0